In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!sudo tar xvf '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import cv2
import sys
from tensorflow.keras.models import load_model 
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16

In [ ]:
data = pd.read_csv('/kaggle/working/fer2013/fer2013.csv')

In [ ]:
images = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
images = np.stack(images.values)
images = images.reshape(images.shape[0], 48, 48, 1)
color_images = []
for img in images:
    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    color_images.append(color_img)

color_images = np.array(color_images)
labels = data['emotion']

In [ ]:
# train / test data split 분할
train_images, test_images, train_labels, test_labels = train_test_split(color_images, labels, test_size=0.1, stratify=labels)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, stratify=train_labels)

# 라벨 one-hot encoding
num_classes = 7
train_labels = to_categorical(train_labels, num_classes)
val_labels = to_categorical(val_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

In [ ]:
from keras.applications.vgg16 import VGG16

vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

In [ ]:
# 모델 구현
model = Sequential()

# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

# 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
import os
MODEL_DIR ='./model/'

if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

modelpath  = './model/{epoch:2d}-{val_loss:.4f}.hdf5'

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience= 5)


checkpoint = keras.callbacks.ModelCheckpoint(filepath = modelpath,
                                             monitor = 'val_loss',
                                             verbos = 1,
                                             save_best_only = True)

batch_size = 256
epochs = 300
# history = model.fit(train_set, train_labels,
#                     batch_size=batch_size, 
#                     epochs=epochs, 
#                     validation_data=(val_images, val_LABEL))
history = model.fit(train_images, train_labels,
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(val_images, val_labels), 
                    callbacks=[early_stopping, checkpoint])

In [ ]:
# 평가
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)